# 특성 공학과 규제

In [1]:
import pandas as pd

df = pd.read_csv('data/Fish.csv')
perch_df = df.loc[df['Species'] == 'Perch']
perch_df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
72,Perch,5.9,7.5,8.4,8.8,2.1120,1.4080
73,Perch,32.0,12.5,13.7,14.7,3.5280,1.9992
74,Perch,40.0,13.8,15.0,16.0,3.8240,2.4320
75,Perch,51.5,15.0,16.2,17.2,4.5924,2.6316
76,Perch,70.0,15.7,17.4,18.5,4.5880,2.9415


In [4]:
perch_full = perch_df[['Length2', 'Height', 'Width']]
perch_weight = perch_df[['Weight']]

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
train_test_split(perch_full, perch_weight)

In [12]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias=False)

In [13]:
poly.fit([[3, 5]])
poly.transform([[3, 5]])

array([[ 3.,  5.,  9., 15., 25.]])

In [14]:
# 가지고 있는 농어의 길이, 두께 등을 특성 변환해보기

poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)

In [17]:
train_poly[:3]

array([[  44.        ,   12.4888    ,    7.5958    , 1936.        ,
         549.5072    ,  334.2152    ,  155.97012544,   94.86242704,
          57.69617764],
       [  25.6       ,    6.561     ,    4.239     ,  655.36      ,
         167.9616    ,  108.5184    ,   43.046721  ,   27.812079  ,
          17.969121  ],
       [  30.        ,    7.6156    ,    4.7716    ,  900.        ,
         228.468     ,  143.148     ,   57.99736336,   36.33859696,
          22.76816656]])

In [18]:
train_poly.shape

(42, 9)

In [20]:
poly.get_feature_names_out()

array(['Length2', 'Height', 'Width', 'Length2^2', 'Length2 Height',
       'Length2 Width', 'Height^2', 'Height Width', 'Width^2'],
      dtype=object)

In [22]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(train_poly, train_target)

lr.score(train_poly, train_target)

0.9874125829265775

In [23]:
test_poly = poly.transform(test_input)

In [24]:
lr.score(test_poly, test_target)

0.9905891310355566

In [27]:
poly = PolynomialFeatures(degree=5, include_bias=False) # degree: 몇 개의 조합을 만들지 결정
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
train_poly.shape

(42, 55)

In [28]:
lr.fit(train_poly, train_target)

LinearRegression()

In [29]:
print(lr.score(train_poly, train_target)) # 학습에 치중한 나머지 채점에 약해짐
print(lr.score(test_poly, test_target))

0.9999999999999961
0.8155788995498732


In [30]:
# 규제

In [33]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_poly)

StandardScaler()

In [34]:
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

In [36]:
train_scaled[:2]

array([[ 1.76567506,  1.53828938,  1.54085869,  2.00795097,  1.84113905,
         1.85219104,  1.6731661 ,  1.68363164,  1.65873623,  2.21868242,
         2.07228787,  2.08882827,  1.92323068,  1.93964657,  1.92630076,
         1.7733149 ,  1.78861893,  1.77633692,  1.73816288,  2.41396428,
         2.27828501,  2.29867814,  2.13925002,  2.16056826,  2.15421108,
         1.99774124,  2.01912857,  2.0142084 ,  1.98421811,  1.85464568,
         1.87527308,  1.87096071,  1.84275913,  1.79278724,  2.59782357,
         2.46916632,  2.49197594,  2.33670397,  2.36154328,  2.35958225,
         2.20091962,  2.22704491,  2.22734308,  2.20292557,  2.06239228,
         2.08899904,  2.09079653,  2.06867277,  2.02443989,  1.92181986,
         1.94807406,  1.95057616,  1.93002222,  1.88805186,  1.82704435],
       [-0.35643021, -0.55040365, -0.40372772, -0.47014496, -0.56270213,
        -0.49506731, -0.63290506, -0.57568014, -0.51227898, -0.54166724,
        -0.59456444, -0.5580192 , -0.6368171 , -0.

In [40]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1)
ridge.fit(train_scaled, train_target)

Ridge(alpha=0.1)

In [43]:
print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

0.9889464349628184
0.9935321911916244


In [44]:
# 라쏘

In [46]:
from sklearn.linear_model import Lasso

lasso = Lasso() # 기본값 1
lasso.fit(train_scaled, train_target)

print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

0.9862920875949489
0.9933936616389513


C:\Users\1-09\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.524e+02, tolerance: 5.070e+02
  model = cd_fast.enet_coordinate_descent(


In [47]:
lasso.coef_

array([ 0.        ,  0.        ,  0.        , 56.54336154,  0.        ,
        2.82091607,  0.        ,  0.        , 87.65159214, 12.55547415,
       22.84023595,  0.        , 49.91773394,  0.        ,  0.        ,
       43.53502931,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , 71.99226363,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso

# 데이터 불러오기
df = pd.read_csv('data/Fish.csv')
perch_df = df.loc[df['Species'] == 'Perch']
perch_full = perch_df[['Length2', 'Height', 'Width']]
perch_weight = perch_df[['Weight']]

# 학습용 채점용 분리하기
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)

# 정규화 (StandardScaler)
scaler = StandardScaler()
train_input_scaled = scaler.fit_transform(train_input)
test_input_scaled = scaler.transform(test_input)

# 다항 회귀 적용: PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False)  # degree를 적절히 설정
train_poly = poly.fit_transform(train_input_scaled)
test_poly = poly.transform(test_input_scaled)

# 모델 학습: Ridge 회귀 (L2 정규화)
ridge = Ridge(alpha=1.0)  # alpha는 정규화 강도 (기본값은 1.0)
ridge.fit(train_poly, train_target)

# 예측: 1, 1 에 대해 예측
scaled_1_1 = scaler.transform([[1, 1, 0]])  # 세 번째 특성인 'Width'를 0으로 설정
poly_1_1 = poly.transform(scaled_1_1)  # 다항 변환
prediction_ridge_1_1 = ridge.predict(poly_1_1)
print(f"Ridge Prediction for (1, 1): {prediction_ridge_1_1}")

# 예측: 3, 9 에 대해 예측
scaled_3_9 = scaler.transform([[3, 9, 0]])  # 세 번째 특성인 'Width'를 0으로 설정
poly_3_9 = poly.transform(scaled_3_9)  # 다항 변환
prediction_ridge_3_9 = ridge.predict(poly_3_9)
print(f"Ridge Prediction for (3, 9): {prediction_ridge_3_9}")

# 모델 학습: Lasso 회귀 (L1 정규화)
lasso = Lasso(alpha=0.1)  # alpha는 정규화 강도 (기본값은 1.0)
lasso.fit(train_poly, train_target)

# 예측: 1, 1 에 대해 예측
prediction_lasso_1_1 = lasso.predict(poly_1_1)
print(f"Lasso Prediction for (1, 1): {prediction_lasso_1_1}")

# 예측: 3, 9 에 대해 예측
prediction_lasso_3_9 = lasso.predict(poly_3_9)
print(f"Lasso Prediction for (3, 9): {prediction_lasso_3_9}")


Ridge Prediction for (1, 1): [137.83836984]
Ridge Prediction for (3, 9): [-23.43368698]
Lasso Prediction for (1, 1): [107.04262655]
Lasso Prediction for (3, 9): [321.08416965]


C:\Users\1-09\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\1-09\Desktop\DAMF2\ml\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
